In [1]:
import gradio as gr
from transformers import AutoImageProcessor
from transformers import SiglipForImageClassification
from transformers.image_utils import load_image
from PIL import Image
import torch

c:\Users\Shruti\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "prithivMLmods/Guard-Against-Unsafe-Content-Siglip2"
model = SiglipForImageClassification.from_pretrained(model_name)
processor = AutoImageProcessor.from_pretrained(model_name)

c:\Users\Shruti\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shruti\.cache\huggingface\hub\models--prithivMLmods--Guard-Against-Unsafe-Content-Siglip2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Using a slow image processor as `use_fast` is unset and a slo

In [3]:
def nsfw_detection(image):
    """Predicts NSFW probability scores for an image."""
    image = Image.fromarray(image).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1).squeeze().tolist()
    
    labels = model.config.id2label
    predictions = {labels[i]: round(probs[i], 3) for i in range(len(probs))}
    
    return predictions

In [ ]:
iface = gr.Interface(
    fn=nsfw_detection,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Label(label="NSFW Content Detection"),
    title="NSFW Image Detection",
    description="Upload an image to check if it contains unsafe content such as vulgarity or nudity."
)

if __name__ == "__main__":
    iface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Shruti\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Shruti\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Shruti\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\blocks.py", line 2042, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Shruti\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\blocks.py", line 1589, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Shruti\AppData\Lo

In [15]:
from google.cloud import vision
import io

# Initialize client
client = vision.ImageAnnotatorClient()

# Load image
with io.open("C:\\Users\\Shruti\\Downloads\\_- visual selection (6).png", "rb") as image_file:
    content = image_file.read()

image = vision.Image(content=content)

# Perform Safe Search Detection
response = client.safe_search_detection(image=image)
safe = response.safe_search_annotation

# Result mapping
likelihood = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE', 'LIKELY', 'VERY_LIKELY')

# Print results
print(f"Adult: {likelihood[safe.adult]}")
print(f"Racy: {likelihood[safe.racy]}")
print(f"Violence: {likelihood[safe.violence]}")
print(f"Medical: {likelihood[safe.medical]}")
print(f"Spoof: {likelihood[safe.spoof]}")

# Decision logic
if safe.adult >= 3 or safe.racy >= 3:  # POSSIBLE or higher
    print("🚫 Unsafe Content Detected")
else:
    print("✅ Content is Safe")


Adult: VERY_UNLIKELY
Racy: VERY_UNLIKELY
Violence: VERY_UNLIKELY
Medical: VERY_UNLIKELY
Spoof: VERY_UNLIKELY
✅ Content is Safe
